# Introduction to Network Forensics

**Scenario:** You are a cybersecurity analyst working in the Security Operations Center (SOC) of BookWorld, an expansive online bookstore renowned for its vast selection of literature. BookWorld prides itself on providing a seamless and secure shopping experience for book enthusiasts around the globe. Recently, you've been tasked with reinforcing the company's cybersecurity posture, monitoring network traffic, and ensuring that the digital environment remains safe from threats.

Late one evening, an automated alert is triggered by an unusual spike in database queries and server resource usage, indicating potential malicious activity. This anomaly raises concerns about the integrity of BookWorld's customer data and internal systems, prompting an immediate and thorough investigation.

As the lead analyst on this case, you are required to analyze the network traffic to uncover the nature of the suspicious activity. Your objectives include identifying the attack vector, assessing the scope of any potential data breach, and determining if the attacker gained further access to BookWorld's internal systems.

**Tools to Use:** Tshark, Wireshark, Network Miner

In [ ]:
!apt update -y
!apt install tshark -y

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [2,542 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,183 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/univers

The command **!apt update -y** is used in many interactive environments like Jupyter Notebooks or Google Colab to execute shell commands. Here, the exclamation mark (!) indicates that the command should be run in the system's command line (internally deployed docker from Google Collabs), rather than within the notebook itself.

The apt tool is a package management utility for Debian-based Linux distributions such as Ubuntu. The update option tells apt to refresh the list of available software packages and their versions. This is an essential step to ensure the package manager has the most up-to-date information. The -y flag automatically answers "yes" to any prompts that might arise during the execution, making the process non-interactive and suitable for automated environments.

The install option tells apt to install a specified package, in this case, tshark. TShark is a powerful network protocol analyzer that captures network packets and displays detailed information about them.

**Quest 01:** By knowing the attacker's IP, we can analyze all logs and actions related to that IP and determine the extent of the attack, the duration of the attack, and the techniques used. Can you provide the attacker's IP?

Let's execute our first command as follows. The following command reads TCP packets (from a pcap file using the -r argument), extracts the source IP addresses (-T fields -e ip.src), and then counts and sorts these addresses by frequency ( | sort). The result is a list of the unique source IP addresses (-q argument removes the duplicates.

In [ ]:
!tshark -r WebInvestigation.pcap -Y "tcp" -T fields -e ip.src | sort | uniq -c | sort -nr

Running as user "root" and group "root". This could be dangerous.
tshark: The file "WebInvestigation.pcap" doesn't exist.


There are three IP addresses. One of them is the attacker and the other is the BookWorld server. Check the Network Miner by opening the pcap file and check the host details for each one of them.

In [ ]:
#!tshark -r WebInvestigation.pcap -Y "http"

The above command helps to identify and display all HTTP-related packets captured in the PCAP file. You can see a lot of 404 errors. A huge number of 404 errors is an indication of abnormal or malicious activity.

**Quest 02:** Now...We got the malicious IP, didn't we? If the geographical origin of an IP address is known to be from a region that has no business or expected traffic with our network, this can be an indicator of a targeted attack. Can you determine the origin city of the attacker? Which is the country of BookWorld?

---

IP geolocation allows you to determine the geographic location associated with an IP address. Several websites and services offer this functionality, including IP2Location, MaxMind, IPinfo.io, WhatIsMyIPAddress.com, and IPGeolocation.io. You can input an IP address into these platforms to retrieve information such as the country, region, city, latitude, and longitude associated with that IP address. While IP geolocation can be useful, it's important to remember that it may not always be 100% accurate due to various factors such as dynamic IP assignments and the use of proxies or VPNs.

The curl comman below is used to fetch data from a URL, and in this case, it's fetching JSON data from https://ipapi.co/73.124.22.98/json. This URL corresponds to the IP geolocation service provided by ipapi.co. The JSON data returned will contain information about the geolocation of the IP address 73.124.22.98, such as its country, city, latitude, longitude, and more. JSON (JavaScript Object Notation) is a lightweight data interchange format commonly used for transmitting data between a server and a client.

In [ ]:
#!curl https://ipapi.co/73.124.22.98/json
#!curl https://ipapi.co/111.224.250.131/json
#!curl https://ipapi.co/170.40.150.126/json

{
    "ip": "73.124.22.98",
    "network": "73.124.22.0/23",
    "version": "IPv4",
    "city": "Miami",
    "region": "Florida",
    "region_code": "FL",
    "country": "US",
    "country_name": "United States",
    "country_code": "US",
    "country_code_iso3": "USA",
    "country_capital": "Washington",
    "country_tld": ".us",
    "continent_code": "NA",
    "in_eu": false,
    "postal": "33165",
    "latitude": 25.7277,
    "longitude": -80.3716,
    "timezone": "America/New_York",
    "utc_offset": "-0400",
    "country_calling_code": "+1",
    "currency": "USD",
    "currency_name": "Dollar",
    "languages": "en-US,es-US,haw,fr",
    "country_area": 9629091.0,
    "country_population": 327167434,
    "asn": "AS7922",
    "org": "COMCAST-7922"
}

**Quest 03:** Identifying the exploited script allows security teams to understand exactly which vulnerability was used in the attack. This knowledge is critical for finding the appropriate patch or workaround to close the security gap and prevent future exploitation. Can you provide the vulnerable script name?

Hint: We can see on Network Minter what software tools the malicious actor used. Tshark also responds with information, the final column indicates the potential software applications which were used. You sure can see a lot of similar entries. That's because the malicioud actor conducted a dictionary (bruteforce) attack to discover all the subfolders or subpages of the website. There is also another tool which was used, this relates to MySQL.

In [ ]:
!tshark -r WebInvestigation.pcap -Y "http.request.method == POST" | uniq -c

Running as user "root" and group "root". This could be dangerous.
      1 88664 2023.126774 111.224.250.131 → 73.124.22.98 HTTP 655 POST /admin/login.php HTTP/1.1  (application/x-www-form-urlencoded)
      1 88677 2069.847495 111.224.250.131 → 73.124.22.98 HTTP 658 POST /admin/login.php HTTP/1.1  (application/x-www-form-urlencoded)
      1 88681 2073.888784 111.224.250.131 → 73.124.22.98 HTTP 659 POST /admin/login.php HTTP/1.1  (application/x-www-form-urlencoded)
      1 88699 2294.305526 111.224.250.131 → 73.124.22.98 HTTP 661 POST /admin/login.php HTTP/1.1  (application/x-www-form-urlencoded)
      1 88757 2697.157173 111.224.250.131 → 73.124.22.98 HTTP 1122 POST /admin/index.php HTTP/1.1  (application/x-php)


As you can see there are a lot of fields (columns) in the pcap file. An incremental number, the franme number, source ip and destination ip, the type of the method (POST or GET) and other details. The final field though provides the

In [ ]:
!tshark -r WebInvestigation.pcap -Y "http.request.method == GET" -T fields -e http.user_agent | sort | uniq -c

Running as user "root" and group "root". This could be dangerous.
  41527 gobuster/3.6
     25 Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36
     31 Mozilla/5.0 (X11; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/115.0
    121 sqlmap/1.8.3#stable (https://sqlmap.org)


In [ ]:
!tshark -r WebInvestigation.pcap -Y "http.request" -T fields -e frame.number -e ip.src -e ip.dst -e http.request.method -e http.host -e http.request.uri -e http.user_agent | sort | uniq -c | sort -nr

Streaming output truncated to the last 5000 lines.
      1 41539	111.224.250.131	73.124.22.98	GET	bookworldstore.com	/irc.axd	gobuster/3.6
      1 41538	111.224.250.131	73.124.22.98	GET	bookworldstore.com	/irc.js	gobuster/3.6
      1 41537	111.224.250.131	73.124.22.98	GET	bookworldstore.com	/irc	gobuster/3.6
      1 41536	111.224.250.131	73.124.22.98	GET	bookworldstore.com	/iraq.cgi	gobuster/3.6
      1 41535	111.224.250.131	73.124.22.98	GET	bookworldstore.com	/iraq.bak	gobuster/3.6
      1 41534	111.224.250.131	73.124.22.98	GET	bookworldstore.com	/iraq.asp	gobuster/3.6
      1 41533	111.224.250.131	73.124.22.98	GET	bookworldstore.com	/iraq.html	gobuster/3.6
      1 41532	111.224.250.131	73.124.22.98	GET	bookworldstore.com	/iraq.txt	gobuster/3.6
      1 4153	111.224.250.131	73.124.22.98	GET	bookworldstore.com	/0.asp	gobuster/3.6
      1 41530	111.224.250.131	73.124.22.98	GET	bookworldstore.com	/ir.asp	gobuster/3.6
      1 41513	111.224.250.131	73.124.22.98	GET	bookworldstore.com	/iraq.

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Now let's see... This scirpt it might be from the company but maybe somehow the attacker uploaded the malicious script discovered above.If you can check and filter the pcap to see what kind of activity was executed using this .php script? The HTTP GET and POST methods usually refer to such activities. Oh, and consider that here is a very common attack named SQL Injection.

In [ ]:
!tshark -r WebInvestigation.pcap -Y "http.request and (http.request.uri matches \"(?i)(SELECT|INSERT|UPDATE|DELETE|WHERE|FROM|UNION)\")"

Running as user "root" and group "root". This could be dangerous.
  426 1692.547971 111.224.250.131 → 73.124.22.98 HTTP 538 GET /search.php?search=book&ZscL=7696%20AND%201%3D1%20UNION%20ALL%20SELECT%201%2CNULL%2C%27%3Cscript%3Ealert%28%22XSS%22%29%3C%2Fscript%3E%27%2Ctable_name%20FROM%20information_schema.tables%20WHERE%202%3E1--%2F%2A%2A%2F%3B%20EXEC%20xp_cmdshell%28%27cat%20..%2F..%2F..%2Fetc%2Fpasswd%27%29%23 HTTP/1.1 
  567 1693.111684 111.224.250.131 → 73.124.22.98 HTTP 401 GET /search.php?search=%28SELECT%20%28CASE%20WHEN%20%285752%3D1273%29%20THEN%20%27book%27%20ELSE%20%28SELECT%201273%20UNION%20SELECT%204191%29%20END%29%29 HTTP/1.1 
  578 1693.113285 111.224.250.131 → 73.124.22.98 HTTP 401 GET /search.php?search=%28SELECT%20%28CASE%20WHEN%20%288006%3D8006%29%20THEN%20%27book%27%20ELSE%20%28SELECT%203826%20UNION%20SELECT%201335%29%20END%29%29 HTTP/1.1 
  588 1693.116106 111.224.250.131 → 73.124.22.98 HTTP 440 GET /search.php?search=book%27%29%20AND%20EXTRACTVALUE%282785%2CCONCAT

**Quest 04:** Establishing the timeline of an attack, starting from the initial exploitation attempt, What's the complete request URI of the first SQLi attempt by the attacker?

In [ ]:
!tshark -r WebInvestigation.pcap -Y "http.request and (http.request.uri matches \"(?i)(1=1)\")" -T fields -e frame.number -e ip.src -e ip.dst -e http.request.method -e http.host -e http.request.uri -e http.user_agent

Running as user "root" and group "root". This could be dangerous.
357	111.224.250.131	73.124.22.98	GET	bookworldstore.com	/search.php?search=book%20and%201=1;%20--%20-	Mozilla/5.0 (X11; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/115.0
379	111.224.250.131	73.124.22.98	GET	bookworldstore.com	/search.php?search=test%E2%80%99%20OR%201=1;%20--	Mozilla/5.0 (X11; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/115.0
389	111.224.250.131	73.124.22.98	GET	bookworldstore.com	/search.php?search=%27%20or%201=1;%20--%20-	Mozilla/5.0 (X11; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/115.0
404	111.224.250.131	73.124.22.98	GET	bookworldstore.com	/search.php?search=%27%20or%201=1;%20--%20-	Mozilla/5.0 (X11; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/115.0


**Quest 05:** Can you provide the complete request URI that was used to read the web server available databases?

Hint: Use NetworkMiner going to files and list by size. Put as a keyword search.php. Check some of the queries of the largest size 200-800B. Inspect carefully manually all the relevant responses which provide usernames, emails etc. Search for the request that is related to database enumeration, finding the names of the databases. Then keep not of the specific Frame id. You can extract the parameters from the parameters tab of the specific frame id. You can also navigate using Wireshark.


**Quest 06:** Assessing the impact of the breach and data access is crucial, including the potential harm to the organization's reputation. What's the table name containing the website users data?

Hint: Search again as before but now maybe its a larger packet as a respose from search.php query. Inside the query that provided usernames and emails you will find the table_name where the SELECT .... FROM ... SQL query was exectuted. On Network Miner -> Copy Parameter Values on the tab with the parameters. The solution is only the name of the table.

**Quest 07:** The website directories hidden from the public could serve as an unauthorized access point or contain sensitive functionalities not intended for public access. Can you provide name of the directory discovered by the attacker?

Hint: Search on the crendentials tab.

In [ ]:
!tshark -r WebInvestigation.pcap -Y "http.request" -T fields -e http.request.uri | grep -Eo '^\/[^?]*\/$' | sort | uniq

Running as user "root" and group "root". This could be dangerous.
/admin/
/admin/uploads/
/cgi-bin/


**Quest 08:** Knowing which credentials were used allows us to determine the extent of account compromise. What's the credentials used by the attacker for logging in?

Hint: Check the Credentials tab or find the relevant packet frame number.



In [ ]:
!tshark -r WebInvestigation.pcap -Y "http.request.method == POST"

Running as user "root" and group "root". This could be dangerous.
88664 2023.126774 111.224.250.131 → 73.124.22.98 HTTP 655 POST /admin/login.php HTTP/1.1  (application/x-www-form-urlencoded)
88677 2069.847495 111.224.250.131 → 73.124.22.98 HTTP 658 POST /admin/login.php HTTP/1.1  (application/x-www-form-urlencoded)
88681 2073.888784 111.224.250.131 → 73.124.22.98 HTTP 659 POST /admin/login.php HTTP/1.1  (application/x-www-form-urlencoded)
88699 2294.305526 111.224.250.131 → 73.124.22.98 HTTP 661 POST /admin/login.php HTTP/1.1  (application/x-www-form-urlencoded)
88757 2697.157173 111.224.250.131 → 73.124.22.98 HTTP 1122 POST /admin/index.php HTTP/1.1  (application/x-php)


**Quest 09:** We need to determine if the attacker gained further access or control on our web server. What's the name of the malicious script uploaded by the attacker?

Hint: Search for POST methods since it is an upload. It should be in the final network frames using the /search.php as the standard attack vector.

